In [1]:
import os

from sklearn.feature_extraction.text import TfidfVectorizer

from knodle.model import LogisticRegressionModel
from joblib import load, dump
import pandas as pd
from torch import Tensor

from knodle.trainer import KnnDenoising
import logging
import torch

# Imports
from knodle.trainer.baseline.baseline import SimpleDsModelTrainer
from knodle.model import LogisticRegressionModel
from knodle.trainer.config.TrainerConfig import TrainerConfig
from snorkel.classification import cross_entropy_with_probs
from torch.optim import SGD
import numpy as np
logger = logging.getLogger(__name__)

OUTPUT_CLASSES = 2

os.chdir('/Users/sandro/repo/knodle/')


2020-11-24 10:45:26,605 root         INFO     Initalized logger


In [2]:
# Helper functions

def read_evaluation_data():
    imdb_dataset = pd.read_csv("tutorials/ImdbDataset/imdb_data_preprocessed.csv")
    applied_lfs = load("tutorials/ImdbDataset/applied_lfs.lib")
    return imdb_dataset, applied_lfs


def create_tfidf_values(text_data: [str]):
    if os.path.exists("tutorials/ImdbDataset/tfidf.lib"):
        cached_data = load("tutorials/ImdbDataset/tfidf.lib")
        if cached_data.shape == text_data.shape:
            return cached_data
    logger.info("Create new")

    vectorizer = TfidfVectorizer()
    transformed_data = vectorizer.fit_transform(text_data)
    dump(transformed_data, "tutorials/ImdbDataset/tfidf.lib")
    return transformed_data


def binary_accuracy(y_pred, y_true):
    """Function to calculate binary accuracy per batch"""
    y_true = y_true if len(y_true.shape) == 1 else torch.argmax(y_true, dim=-1)
    y_pred_max = torch.argmax(y_pred, dim=-1)
    correct_pred = (y_pred_max == y_true).float()
    acc = correct_pred.sum() / len(correct_pred)
    return acc

def test(model, test_features, test_labels):
    model.eval()

    predictions = model(test_features)

    acc = binary_accuracy(predictions, test_labels)
    print(acc.numpy())
    

# Evaluation example


In [3]:
imdb_dataset, rule_matches = read_evaluation_data()
imdb_sample = imdb_dataset.sample(1000, random_state=123)
rule_matches_sample = rule_matches[imdb_sample.index]
assert len(imdb_sample) == len(rule_matches_sample)

tfidf_values = create_tfidf_values(imdb_sample.reviews_preprocessed.values)

tfidf_tensor = Tensor(tfidf_values.toarray())


train_tfidf = tfidf_values[:800]
train_tensor = Tensor(train_tfidf.toarray())
train_rule_matches = rule_matches_sample[:800]

test_tfidf = tfidf_values[801:]
test_tensor = Tensor(test_tfidf.toarray())


test_label_tensor = Tensor(imdb_sample[801:].label_id.values)

2020-11-24 10:46:35,113 __main__     INFO     Create new


## Baseline Model

In [4]:
model = LogisticRegressionModel(train_tensor.shape[1], OUTPUT_CLASSES)
trainer_config = TrainerConfig(model, criterion=cross_entropy_with_probs, optimizer_=SGD(model.parameters(), lr=5))

trainer = SimpleDsModelTrainer(model, trainer_config=trainer_config)
trainer.train(inputs=train_tensor, rule_matches=train_rule_matches, epochs=1000)

2020-11-24 10:47:05,127 knodle.trainer.ds_model_trainer.ds_model_trainer INFO     Initalized trainer with custom model config: {'criterion': <function cross_entropy_with_probs at 0x1418e22f0>, 'batch_size': 32, 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 5
    momentum: 0
    nesterov: False
    weight_decay: 0
), 'output_classes': 2}
2020-11-24 10:47:05,158 knodle.trainer.baseline.baseline INFO     ======================================
2020-11-24 10:47:05,158 knodle.trainer.baseline.baseline INFO     Training starts
2020-11-24 10:47:05,159 knodle.trainer.baseline.baseline INFO     ======================================
2020-11-24 10:47:05,160 knodle.trainer.baseline.baseline INFO     Epoch: 0
2020-11-24 10:47:05,229 knodle.trainer.baseline.baseline INFO     Loss is: 0.6968994140625
2020-11-24 10:47:05,256 knodle.trainer.baseline.baseline INFO     Epoch: 1
2020-11-24 10:47:05,266 knodle.trainer.baseline.baseline INFO     Loss is: 0.6121888160705566
2020-11-24 10:47:0

2020-11-24 10:47:06,395 knodle.trainer.baseline.baseline INFO     Loss is: 0.3778012990951538
2020-11-24 10:47:06,406 knodle.trainer.baseline.baseline INFO     Epoch: 45
2020-11-24 10:47:06,419 knodle.trainer.baseline.baseline INFO     Loss is: 0.3762974143028259
2020-11-24 10:47:06,434 knodle.trainer.baseline.baseline INFO     Epoch: 46
2020-11-24 10:47:06,446 knodle.trainer.baseline.baseline INFO     Loss is: 0.3748052716255188
2020-11-24 10:47:06,461 knodle.trainer.baseline.baseline INFO     Epoch: 47
2020-11-24 10:47:06,471 knodle.trainer.baseline.baseline INFO     Loss is: 0.37332475185394287
2020-11-24 10:47:06,483 knodle.trainer.baseline.baseline INFO     Epoch: 48
2020-11-24 10:47:06,496 knodle.trainer.baseline.baseline INFO     Loss is: 0.3718557357788086
2020-11-24 10:47:06,509 knodle.trainer.baseline.baseline INFO     Epoch: 49
2020-11-24 10:47:06,524 knodle.trainer.baseline.baseline INFO     Loss is: 0.3703981339931488
2020-11-24 10:47:06,536 knodle.trainer.baseline.baselin

2020-11-24 10:47:07,608 knodle.trainer.baseline.baseline INFO     Epoch: 93
2020-11-24 10:47:07,621 knodle.trainer.baseline.baseline INFO     Loss is: 0.3163360059261322
2020-11-24 10:47:07,635 knodle.trainer.baseline.baseline INFO     Epoch: 94
2020-11-24 10:47:07,646 knodle.trainer.baseline.baseline INFO     Loss is: 0.3153083622455597
2020-11-24 10:47:07,659 knodle.trainer.baseline.baseline INFO     Epoch: 95
2020-11-24 10:47:07,671 knodle.trainer.baseline.baseline INFO     Loss is: 0.31428852677345276
2020-11-24 10:47:07,683 knodle.trainer.baseline.baseline INFO     Epoch: 96
2020-11-24 10:47:07,694 knodle.trainer.baseline.baseline INFO     Loss is: 0.31327638030052185
2020-11-24 10:47:07,708 knodle.trainer.baseline.baseline INFO     Epoch: 97
2020-11-24 10:47:07,719 knodle.trainer.baseline.baseline INFO     Loss is: 0.3122718930244446
2020-11-24 10:47:07,732 knodle.trainer.baseline.baseline INFO     Epoch: 98
2020-11-24 10:47:07,744 knodle.trainer.baseline.baseline INFO     Loss i

2020-11-24 10:47:08,735 knodle.trainer.baseline.baseline INFO     Epoch: 141
2020-11-24 10:47:08,746 knodle.trainer.baseline.baseline INFO     Loss is: 0.27464380860328674
2020-11-24 10:47:08,758 knodle.trainer.baseline.baseline INFO     Epoch: 142
2020-11-24 10:47:08,769 knodle.trainer.baseline.baseline INFO     Loss is: 0.273918092250824
2020-11-24 10:47:08,782 knodle.trainer.baseline.baseline INFO     Epoch: 143
2020-11-24 10:47:08,791 knodle.trainer.baseline.baseline INFO     Loss is: 0.2731972932815552
2020-11-24 10:47:08,804 knodle.trainer.baseline.baseline INFO     Epoch: 144
2020-11-24 10:47:08,815 knodle.trainer.baseline.baseline INFO     Loss is: 0.27248141169548035
2020-11-24 10:47:08,826 knodle.trainer.baseline.baseline INFO     Epoch: 145
2020-11-24 10:47:08,838 knodle.trainer.baseline.baseline INFO     Loss is: 0.2717704474925995
2020-11-24 10:47:08,850 knodle.trainer.baseline.baseline INFO     Epoch: 146
2020-11-24 10:47:08,859 knodle.trainer.baseline.baseline INFO     L

2020-11-24 10:47:09,901 knodle.trainer.baseline.baseline INFO     Epoch: 189
2020-11-24 10:47:09,916 knodle.trainer.baseline.baseline INFO     Loss is: 0.24470341205596924
2020-11-24 10:47:09,929 knodle.trainer.baseline.baseline INFO     Epoch: 190
2020-11-24 10:47:09,944 knodle.trainer.baseline.baseline INFO     Loss is: 0.2441718429327011
2020-11-24 10:47:09,956 knodle.trainer.baseline.baseline INFO     Epoch: 191
2020-11-24 10:47:09,967 knodle.trainer.baseline.baseline INFO     Loss is: 0.24364350736141205
2020-11-24 10:47:09,984 knodle.trainer.baseline.baseline INFO     Epoch: 192
2020-11-24 10:47:09,998 knodle.trainer.baseline.baseline INFO     Loss is: 0.24311843514442444
2020-11-24 10:47:10,014 knodle.trainer.baseline.baseline INFO     Epoch: 193
2020-11-24 10:47:10,024 knodle.trainer.baseline.baseline INFO     Loss is: 0.24259647727012634
2020-11-24 10:47:10,040 knodle.trainer.baseline.baseline INFO     Epoch: 194
2020-11-24 10:47:10,050 knodle.trainer.baseline.baseline INFO   

...

2020-11-24 10:47:24,165 knodle.trainer.baseline.baseline INFO     Epoch: 765
2020-11-24 10:47:24,177 knodle.trainer.baseline.baseline INFO     Loss is: 0.13708466291427612
2020-11-24 10:47:24,191 knodle.trainer.baseline.baseline INFO     Epoch: 766
2020-11-24 10:47:24,202 knodle.trainer.baseline.baseline INFO     Loss is: 0.13702277839183807
2020-11-24 10:47:24,215 knodle.trainer.baseline.baseline INFO     Epoch: 767
2020-11-24 10:47:24,226 knodle.trainer.baseline.baseline INFO     Loss is: 0.13696102797985077
2020-11-24 10:47:24,242 knodle.trainer.baseline.baseline INFO     Epoch: 768
2020-11-24 10:47:24,253 knodle.trainer.baseline.baseline INFO     Loss is: 0.1368994563817978
2020-11-24 10:47:24,265 knodle.trainer.baseline.baseline INFO     Epoch: 769
2020-11-24 10:47:24,278 knodle.trainer.baseline.baseline INFO     Loss is: 0.13683800399303436
2020-11-24 10:47:24,289 knodle.trainer.baseline.baseline INFO     Epoch: 770
2020-11-24 10:47:24,300 knodle.trainer.baseline.baseline INFO   

2020-11-24 10:47:25,359 knodle.trainer.baseline.baseline INFO     Epoch: 813
2020-11-24 10:47:25,372 knodle.trainer.baseline.baseline INFO     Loss is: 0.13427679240703583
2020-11-24 10:47:25,383 knodle.trainer.baseline.baseline INFO     Epoch: 814
2020-11-24 10:47:25,394 knodle.trainer.baseline.baseline INFO     Loss is: 0.1342216432094574
2020-11-24 10:47:25,408 knodle.trainer.baseline.baseline INFO     Epoch: 815
2020-11-24 10:47:25,418 knodle.trainer.baseline.baseline INFO     Loss is: 0.1341666281223297
2020-11-24 10:47:25,429 knodle.trainer.baseline.baseline INFO     Epoch: 816
2020-11-24 10:47:25,444 knodle.trainer.baseline.baseline INFO     Loss is: 0.13411171734333038
2020-11-24 10:47:25,455 knodle.trainer.baseline.baseline INFO     Epoch: 817
2020-11-24 10:47:25,465 knodle.trainer.baseline.baseline INFO     Loss is: 0.134056955575943
2020-11-24 10:47:25,479 knodle.trainer.baseline.baseline INFO     Epoch: 818
2020-11-24 10:47:25,488 knodle.trainer.baseline.baseline INFO     L

2020-11-24 10:47:26,535 knodle.trainer.baseline.baseline INFO     Epoch: 861
2020-11-24 10:47:26,548 knodle.trainer.baseline.baseline INFO     Loss is: 0.1317681074142456
2020-11-24 10:47:26,560 knodle.trainer.baseline.baseline INFO     Epoch: 862
2020-11-24 10:47:26,571 knodle.trainer.baseline.baseline INFO     Loss is: 0.1317187249660492
2020-11-24 10:47:26,585 knodle.trainer.baseline.baseline INFO     Epoch: 863
2020-11-24 10:47:26,596 knodle.trainer.baseline.baseline INFO     Loss is: 0.13166941702365875
2020-11-24 10:47:26,609 knodle.trainer.baseline.baseline INFO     Epoch: 864
2020-11-24 10:47:26,619 knodle.trainer.baseline.baseline INFO     Loss is: 0.13162022829055786
2020-11-24 10:47:26,631 knodle.trainer.baseline.baseline INFO     Epoch: 865
2020-11-24 10:47:26,643 knodle.trainer.baseline.baseline INFO     Loss is: 0.13157115876674652
2020-11-24 10:47:26,655 knodle.trainer.baseline.baseline INFO     Epoch: 866
2020-11-24 10:47:26,665 knodle.trainer.baseline.baseline INFO    

2020-11-24 10:47:27,733 knodle.trainer.baseline.baseline INFO     Epoch: 909
2020-11-24 10:47:27,746 knodle.trainer.baseline.baseline INFO     Loss is: 0.12951546907424927
2020-11-24 10:47:27,760 knodle.trainer.baseline.baseline INFO     Epoch: 910
2020-11-24 10:47:27,768 knodle.trainer.baseline.baseline INFO     Loss is: 0.12947097420692444
2020-11-24 10:47:27,783 knodle.trainer.baseline.baseline INFO     Epoch: 911
2020-11-24 10:47:27,796 knodle.trainer.baseline.baseline INFO     Loss is: 0.12942662835121155
2020-11-24 10:47:27,809 knodle.trainer.baseline.baseline INFO     Epoch: 912
2020-11-24 10:47:27,823 knodle.trainer.baseline.baseline INFO     Loss is: 0.12938232719898224
2020-11-24 10:47:27,833 knodle.trainer.baseline.baseline INFO     Epoch: 913
2020-11-24 10:47:27,846 knodle.trainer.baseline.baseline INFO     Loss is: 0.12933816015720367
2020-11-24 10:47:27,860 knodle.trainer.baseline.baseline INFO     Epoch: 914
2020-11-24 10:47:27,869 knodle.trainer.baseline.baseline INFO  

2020-11-24 10:47:28,936 knodle.trainer.baseline.baseline INFO     Epoch: 957
2020-11-24 10:47:28,948 knodle.trainer.baseline.baseline INFO     Loss is: 0.1274833083152771
2020-11-24 10:47:28,962 knodle.trainer.baseline.baseline INFO     Epoch: 958
2020-11-24 10:47:28,972 knodle.trainer.baseline.baseline INFO     Loss is: 0.1274431049823761
2020-11-24 10:47:28,985 knodle.trainer.baseline.baseline INFO     Epoch: 959
2020-11-24 10:47:28,997 knodle.trainer.baseline.baseline INFO     Loss is: 0.12740297615528107
2020-11-24 10:47:29,009 knodle.trainer.baseline.baseline INFO     Epoch: 960
2020-11-24 10:47:29,022 knodle.trainer.baseline.baseline INFO     Loss is: 0.1273629367351532
2020-11-24 10:47:29,033 knodle.trainer.baseline.baseline INFO     Epoch: 961
2020-11-24 10:47:29,046 knodle.trainer.baseline.baseline INFO     Loss is: 0.1273229718208313
2020-11-24 10:47:29,059 knodle.trainer.baseline.baseline INFO     Epoch: 962
2020-11-24 10:47:29,068 knodle.trainer.baseline.baseline INFO     L

In [5]:
test(trainer.model, test_features=test_tensor, test_labels=test_label_tensor)

0.48241207


## kNN Model

In [6]:

model = LogisticRegressionModel(train_tensor.shape[1], OUTPUT_CLASSES)

trainer_config = TrainerConfig(model, criterion=cross_entropy_with_probs, optimizer_=SGD(model.parameters(), lr=5))

trainer = KnnDenoising(model, trainer_config=trainer_config)
trainer.train(inputs=train_tensor, rule_matches=train_rule_matches, epochs=1000, tfidf_values=train_tfidf, k=4)


2020-11-24 10:48:06,566 knodle.trainer.ds_model_trainer.ds_model_trainer INFO     Initalized trainer with custom model config: {'criterion': <function cross_entropy_with_probs at 0x1418e22f0>, 'batch_size': 32, 'optimizer': SGD (
Parameter Group 0
    dampening: 0
    lr: 5
    momentum: 0
    nesterov: False
    weight_decay: 0
), 'output_classes': 2}
2020-11-24 10:48:06,567 knodle.trainer.knn_denoising.knn_denoising INFO     Start denoising labeling functions with k: 4.
2020-11-24 10:48:06,567 knodle.trainer.knn_denoising.knn_denoising INFO     This can take a while ...
800it [00:00, 29692.35it/s]
2020-11-24 10:48:06,755 knodle.trainer.knn_denoising.knn_denoising INFO     ======================================
2020-11-24 10:48:06,756 knodle.trainer.knn_denoising.knn_denoising INFO     Training starts
2020-11-24 10:48:06,756 knodle.trainer.knn_denoising.knn_denoising INFO     ======================================
2020-11-24 10:48:06,757 knodle.trainer.knn_denoising.knn_denoising INFO

2020-11-24 10:48:07,745 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2904021739959717
2020-11-24 10:48:07,761 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 39
2020-11-24 10:48:07,774 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2894112169742584
2020-11-24 10:48:07,787 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 40
2020-11-24 10:48:07,800 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2884247303009033
2020-11-24 10:48:07,814 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 41
2020-11-24 10:48:07,828 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2874426543712616
2020-11-24 10:48:07,841 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 42
2020-11-24 10:48:07,849 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.286465048789978
2020-11-24 10:48:07,864 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 43
2020-11-24 10:48:07,877 knodle.trainer.knn_denoisin

2020-11-24 10:48:08,828 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 82
2020-11-24 10:48:08,837 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2509491443634033
2020-11-24 10:48:08,849 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 83
2020-11-24 10:48:08,862 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.25014883279800415
2020-11-24 10:48:08,875 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 84
2020-11-24 10:48:08,885 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.24935263395309448
2020-11-24 10:48:08,899 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 85
2020-11-24 10:48:08,910 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.24856054782867432
2020-11-24 10:48:08,922 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 86
2020-11-24 10:48:08,935 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.24777254462242126
2020-11-24 10:48:08,948 knodle.trainer.knn_den

2020-11-24 10:48:09,887 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 125
2020-11-24 10:48:09,899 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.22008216381072998
2020-11-24 10:48:09,912 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 126
2020-11-24 10:48:09,925 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.21944576501846313
2020-11-24 10:48:09,938 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 127
2020-11-24 10:48:09,946 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.21881279349327087
2020-11-24 10:48:09,958 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 128
2020-11-24 10:48:09,970 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.2181832194328308
2020-11-24 10:48:09,982 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 129
2020-11-24 10:48:09,995 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.21755708754062653
2020-11-24 10:48:10,007 knodle.trainer.kn

2020-11-24 10:48:10,934 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 168
2020-11-24 10:48:10,946 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.19561034440994263
2020-11-24 10:48:10,961 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 169
2020-11-24 10:48:10,972 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.19510628283023834
2020-11-24 10:48:10,984 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 170
2020-11-24 10:48:11,000 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.19460488855838776
2020-11-24 10:48:11,012 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 171
2020-11-24 10:48:11,026 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.19410619139671326
2020-11-24 10:48:11,038 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 172
2020-11-24 10:48:11,049 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.19361014664173126
2020-11-24 10:48:11,064 knodle.trainer.k

...

2020-11-24 10:48:26,545 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 813
2020-11-24 10:48:26,558 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08696159720420837
2020-11-24 10:48:26,571 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 814
2020-11-24 10:48:26,581 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08691182732582092
2020-11-24 10:48:26,594 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 815
2020-11-24 10:48:26,606 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08686217665672302
2020-11-24 10:48:26,617 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 816
2020-11-24 10:48:26,627 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08681263029575348
2020-11-24 10:48:26,640 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 817
2020-11-24 10:48:26,650 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08676321804523468
2020-11-24 10:48:26,663 knodle.trainer.k

2020-11-24 10:48:27,547 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 856
2020-11-24 10:48:27,558 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08492395281791687
2020-11-24 10:48:27,573 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 857
2020-11-24 10:48:27,582 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08487895876169205
2020-11-24 10:48:27,597 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 858
2020-11-24 10:48:27,610 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08483406156301498
2020-11-24 10:48:27,623 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 859
2020-11-24 10:48:27,634 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08478925377130508
2020-11-24 10:48:27,647 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 860
2020-11-24 10:48:27,660 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08474456518888474
2020-11-24 10:48:27,674 knodle.trainer.k

2020-11-24 10:48:28,565 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 899
2020-11-24 10:48:28,576 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08307827264070511
2020-11-24 10:48:28,588 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 900
2020-11-24 10:48:28,598 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08303743600845337
2020-11-24 10:48:28,611 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 901
2020-11-24 10:48:28,622 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.0829966813325882
2020-11-24 10:48:28,635 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 902
2020-11-24 10:48:28,645 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08295603096485138
2020-11-24 10:48:28,658 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 903
2020-11-24 10:48:28,669 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08291544020175934
2020-11-24 10:48:28,681 knodle.trainer.kn

2020-11-24 10:48:29,679 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 942
2020-11-24 10:48:29,690 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08140026032924652
2020-11-24 10:48:29,704 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 943
2020-11-24 10:48:29,717 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08136306703090668
2020-11-24 10:48:29,730 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 944
2020-11-24 10:48:29,743 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.0813259407877922
2020-11-24 10:48:29,754 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 945
2020-11-24 10:48:29,766 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.0812888890504837
2020-11-24 10:48:29,779 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 946
2020-11-24 10:48:29,789 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.08125192672014236
2020-11-24 10:48:29,801 knodle.trainer.knn

2020-11-24 10:48:30,744 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 985
2020-11-24 10:48:30,754 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07986936718225479
2020-11-24 10:48:30,767 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 986
2020-11-24 10:48:30,781 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07983537018299103
2020-11-24 10:48:30,795 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 987
2020-11-24 10:48:30,806 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07980144768953323
2020-11-24 10:48:30,818 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 988
2020-11-24 10:48:30,833 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07976757735013962
2020-11-24 10:48:30,848 knodle.trainer.knn_denoising.knn_denoising INFO     Epoch: 989
2020-11-24 10:48:30,862 knodle.trainer.knn_denoising.knn_denoising INFO     Loss is: 0.07973378896713257
2020-11-24 10:48:30,876 knodle.trainer.k

In [8]:
test(trainer.model, test_features=test_tensor, test_labels=test_label_tensor)

0.4321608
